Driver at fault Hackathon

In [1]:
!pip install category_encoders

In [14]:
import numpy as np
import pandas as pd
import csv
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
import category_encoders as ce



In [15]:
def scale(a):
    if a < 0:
        a = -a
    if a != 0:
        a /= np.power(10, np.floor(np.log10(a)))
    return a

In [17]:
train_df=pd.read_csv('train.csv')
train_df.head(5)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Report Number                   51490 non-null  object 
 1   Local Case Number               45037 non-null  object 
 2   Agency Name                     51490 non-null  object 
 3   ACRS Report Type                51490 non-null  object 
 4   Crash Date/Time                 51490 non-null  object 
 5   Route Type                      46522 non-null  object 
 6   Road Name                       46988 non-null  object 
 7   Cross-Street Type               46471 non-null  object 
 8   Cross-Street Name               46968 non-null  object 
 9   Off-Road Description            6950 non-null   object 
 10  Municipality                    7567 non-null   object 
 11  Related Non-Motorist            3648 non-null   object 
 12  Collision Type                  

In [18]:
test_df=pd.read_csv('test.csv')
test_df.head(5)
test_df.info()

<ipython-input-18-d3efc7bb4b27>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df=pd.read_csv('test.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77235 entries, 0 to 77234
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Id                              77235 non-null  int64  
 1   Report Number                   77235 non-null  object 
 2   Local Case Number               67162 non-null  object 
 3   Agency Name                     77235 non-null  object 
 4   ACRS Report Type                77235 non-null  object 
 5   Crash Date/Time                 77235 non-null  object 
 6   Route Type                      69768 non-null  object 
 7   Road Name                       70166 non-null  object 
 8   Cross-Street Type               69755 non-null  object 
 9   Cross-Street Name               70164 non-null  object 
 10  Off-Road Description            7069 non-null   object 
 11  Municipality                    8498 non-null   object 
 12  Related Non-Motorist            

In [21]:
temp_train_df=train_df.drop(columns=['Fault'])
temp_test_df=test_df.drop(columns=['Id'])

total_df=pd.concat([temp_train_df,temp_test_df],ignore_index=True)

# 'Report Number', 'Local Case Number', 'Person ID' are not useful
total_df.drop(columns=['Agency Name','Driverless Vehicle','Road Name','Cross-Street Name','Report Number', 'Local Case Number', 'Person ID','Vehicle Model','Vehicle Make','Vehicle ID'],inplace=True)

# dropping columns with to many NaN values
total_df.dropna(axis=1,thresh=35000,inplace=True)


In [22]:
total_df['Driver Substance Abuse'] = total_df['Driver Substance Abuse'].str.replace("CONTRIBUTED",'PRESENT')
total_df['Driver Substance Abuse'] = total_df['Driver Substance Abuse'].str.replace("COMBINED SUBSTANCE",'COMBINATION')

In [23]:
# print(total_df['Equipment Problems'].value_counts(dropna=False))
total_df['Equipment Problems']=total_df['Equipment Problems'].fillna(value='NO MISUSE')
# total_df['Equipment Problems']=total_df['Equipment Problems'].astype('category').cat.codes

# print(total_df['Vehicle Going Dir'].value_counts(dropna=False))
total_df['Vehicle Going Dir']=total_df['Vehicle Going Dir'].fillna('Unknown')
# total_df['Vehicle Going Dir']=total_df['Vehicle Going Dir'].astype('category').cat.codes

total_df['Vehicle Continuing Dir']=total_df['Vehicle Continuing Dir'].fillna('Unknown')
# total_df['Vehicle Continuing Dir']=total_df['Vehicle Continuing Dir'].astype('category').cat.codes

# print(total_df['Vehicle Movement'].value_counts(dropna=False))
total_df['Vehicle Movement']=total_df['Vehicle Movement'].fillna(value='UNKNOWN')
# total_df['Vehicle Movement']=total_df['Vehicle Movement'].astype('category').cat.codes

# print(total_df['Vehicle Body Type'].value_counts(dropna=False))
total_df['Vehicle Body Type']=total_df['Vehicle Body Type'].fillna(value='PASSENGER CAR')
# total_df['Vehicle Body Type']=total_df['Vehicle Body Type'].astype('category').cat.codes

# print(total_df['Vehicle Second Impact Location'].value_counts(dropna=False))
total_df['Vehicle Second Impact Location']=total_df['Vehicle Second Impact Location'].fillna(value='TWELVE OCLOCK')
# total_df['Vehicle Second Impact Location']=total_df['Vehicle Second Impact Location'].astype('category').cat.codes

total_df['Vehicle First Impact Location']=total_df['Vehicle First Impact Location'].fillna(value='TWELVE OCLOCK')
# total_df['Vehicle First Impact Location']=total_df['Vehicle First Impact Location'].astype('category').cat.codes

# print(total_df['Vehicle Damage Extent'].value_counts(dropna=False))
total_df['Vehicle Damage Extent']=total_df['Vehicle Damage Extent'].fillna(value='DISABLING')
# total_df['Vehicle Damage Extent']=total_df['Vehicle Damage Extent'].astype('category').cat.codes

# print(total_df['Drivers License State'].value_counts(dropna=False))
total_df['Drivers License State']=total_df['Drivers License State'].fillna(value='MD')
# total_df['Drivers License State']=total_df['Drivers License State'].astype('category').cat.codes

# total_df['Injury Severity']=total_df['Injury Severity'].astype('category').cat.codes

# print(total_df['Driver Substance Abuse'].value_counts(dropna=False))
total_df['Driver Substance Abuse']=total_df['Driver Substance Abuse'].fillna(value='UNKNOWN')
# total_df['Driver Substance Abuse']=total_df['Driver Substance Abuse'].astype('category').cat.codes

# print(total_df['Traffic Control'].value_counts(dropna=False))
total_df['Traffic Control']=total_df['Traffic Control'].fillna(value='UNKNOWN')
# total_df['Traffic Control']=total_df['Traffic Control'].astype('category').cat.codes

# print(total_df['Light'].value_counts(dropna=False))
total_df['Light']=total_df['Light'].fillna(value='UNKNOWN')
# total_df['Light']=total_df['Light'].astype('category').cat.codes

# print(total_df['Surface Condition'].value_counts(dropna=False))
total_df['Surface Condition']=total_df['Surface Condition'].fillna(value='UNKNOWN')
# total_df['Surface Condition']=total_df['Surface Condition'].astype('category').cat.codes

# print(total_df['Weather'].value_counts(dropna=False))
total_df['Weather']=total_df['Weather'].fillna(value='UNKNOWN')
# total_df['Weather']=total_df['Weather'].astype('category').cat.codes

# print(total_df['Collision Type'].value_counts(dropna=False))
total_df['Collision Type']=total_df['Collision Type'].fillna(value='SAME DIR REAR END')
# total_df['Collision Type']=total_df['Collision Type'].astype('category').cat.codes

# print(total_df['Cross-Street Type'].value_counts(dropna=False))
total_df['Cross-Street Type']=total_df['Cross-Street Type'].fillna(value='Unknown')
# total_df['Cross-Street Type']=total_df['Cross-Street Type'].astype('category').cat.codes

# print(total_df['Route Type'].value_counts(dropna=False))
total_df['Route Type']=total_df['Route Type'].fillna(value='Unknown')
# total_df['Route Type']=total_df['Route Type'].astype('category').cat.codes


In [24]:
total_df['Location']=total_df['Location'].str.replace(',',' ')

split_data = total_df["Location"].str.split(" ")
data = split_data.to_list()


for i in range(len(total_df.index)):
    data[i][1]=float(data[i][1])
    data[i][0]=float(data[i][0])
    data[i][1]=scale(data[i][1])

names = ["Location_1","Location_2"]

temp_df = pd.DataFrame(data, columns=names)

total_df=total_df.drop(columns=['Location'])
total_df=pd.concat([total_df,temp_df],axis=1)

In [25]:

total_df['Crash Date/Time'] = pd.to_datetime(total_df['Crash Date/Time'])
total_df['Hour'] = total_df['Crash Date/Time'].dt.hour
total_df['day'] = total_df['Crash Date/Time'].dt.dayofweek
total_df['year'] = total_df['Crash Date/Time'].dt.year
total_df['month']=total_df['Crash Date/Time'].dt.month
total_df.drop(columns=['Crash Date/Time'],inplace=True)

In [26]:
# one_hot = np.array(pd.get_dummies(total_df))
base_learners = [
                 ('rf_1', RandomForestClassifier(n_estimators=100, max_features='sqrt')),
                 ('rf_2', LGBMClassifier(boosting_type='dart', drop_rate=0.5, max_drop=50, n_estimators=1000)),
                 ('rf_3', XGBClassifier(use_label_encoder=False, learning_rate = 0.14, n_estimators=125, max_depth=8))
                ]

cl=[]
for column in total_df.columns:
    if total_df[column].dtype == object:
        cl.append(column)
encoder = ce.polynomial.PolynomialEncoder(verbose=False, cols=cl, drop_invariant=True)
one_hot= encoder.fit_transform(total_df)

clf = StackingClassifier(estimators=base_learners, n_jobs=12, final_estimator=LGBMClassifier(boosting_type='goss'))

train_labels=np.array(train_df['Fault'])
train_feas=one_hot[:51490]
test_feas=one_hot[51490:]

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/usr/local/lib/python3.10/dist-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


In [27]:
clf.fit(train_feas,train_labels)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


StackingClassifier(estimators=[('rf_1', RandomForestClassifier()),
                               ('rf_2',
                                LGBMClassifier(boosting_type='dart',
                                               drop_rate=0.5, max_drop=50,
                                               n_estimators=1000)),
                               ('rf_3',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metri...
                                              interaction_constraints=None,
                                              learning_rate=0.14, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=8,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=125, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...))],
                   final_estimator=LGBMClassifier(boosting_type='goss'),
                   n_jobs=12)

In [30]:
predicn=clf.predict(test_feas)
predicn=np.array(predicn, dtype=int)
submsn = pd.DataFrame(predicn,columns=['Fault'])
submsn=pd.concat([test_df['Id'],submsn],axis=1)


In [28]:
submsn

,Id,Fault
0,0,1
1,1,0
2,2,0
3,3,1
4,4,1
...,...,...
77230,77230,0
77231,77231,1
77232,77232,0
77233,77233,1


In [29]:


submsn.to_csv('Driver_at_fault_submission.csv',index=False)
